In [1]:
import numpy             as np
import scipy             as sp
import matplotlib.pyplot as plt
import time
from numba import njit

from tqdm            import tqdm
from scipy           import signal

from propagation import received_power, quad_distribute_power, propagation_delay, gen_qrx_onlyclocked

from parametermeasurement import generate_clocks, measure_range_roberts, measure_bearing

from positioning import classicalfix_directbearing_mle, classicalfix_diffbearing_mle, \
                        classicalfix_directrange_mle, classicalfix_diffrange_mle, \
                        runningfix_directbearing_mle, runningfix_directrange_mle

from simulation import Simulation, generate_simulation_clocks

### Set common simulation configurations

In [2]:
iterations = 2000

### world parameters
sim_lmb20 = Simulation(weather        = "clear", 
                 temperature    = 298,   # [K], circuit temperature
                 daynight       = "day_indirectsun", 
                 rxconfig       = "optics/qrx_planoconvex.npz", 
                 txconfig       = "optics/tx_lambertian_20deg_2W.npz", 
                 istxlambertian = True, 
                 f_adc_clk      = 1.0e7,  # [Hz] ADC measurement clock freq
                 f_e            = 1.0e6,  # [Hz] emitted wave freq, left TX
                 f_sim          = 1.0e10) # [Hz] simulation master clock freq

sim_lmb50 = Simulation(weather        = "clear", 
                 temperature    = 298,   # [K], circuit temperature
                 daynight       = "day_indirectsun", 
                 rxconfig       = "optics/qrx_planoconvex.npz", 
                 txconfig       = "optics/tx_lambertian_50deg_2W.npz", 
                 istxlambertian = True, 
                 f_adc_clk      = 1.0e7,  # [Hz] ADC measurement clock freq
                 f_e            = 1.0e6,  # [Hz] emitted wave freq, left TX
                 f_sim          = 1.0e10) # [Hz] simulation master clock freq

sim_pcwlmb = Simulation(weather        = "clear", 
                 temperature    = 298,   # [K], circuit temperature
                 daynight       = "day_indirectsun", 
                 rxconfig       = "optics/qrx_planoconvex.npz", 
                 txconfig       = "optics/tx_piecewiselambertian_2W.npz", 
                 istxlambertian = True, 
                 f_adc_clk      = 1.0e7,  # [Hz] ADC measurement clock freq
                 f_e            = 1.0e6,  # [Hz] emitted wave freq, left TX
                 f_sim          = 1.0e10) # [Hz] simulation master clock freq

sim_real       = Simulation(weather        = "clear", 
                 temperature    = 298,   # [K], circuit temperature
                 daynight       = "day_indirectsun", 
                 rxconfig       = "optics/qrx_planoconvex.npz", 
                 txconfig       = "optics/tx_AgonLbSedan1_2W.npz", 
                 istxlambertian = False, 
                 f_adc_clk      = 1.0e7,  # [Hz] ADC measurement clock freq
                 f_e            = 1.0e6,  # [Hz] emitted wave freq, left TX
                 f_sim          = 1.0e10) # [Hz] simulation master clock freq

### Define full simulation run as a function

In [3]:
def simrun(sim, z, iterations):
    
    L_ego    = 1.6 # [m]
    L_target = 1.5 # [m] arbitrary selection, just to make it different than the ego. This is not known, so it's not used in the algo
    
    ### build lane change trajectory similar to the PIMRC article
    # starts with constant heading, sliding, then starts to level out with changing heading (a curved portion at the end on xy plane)
    a = np.load('trajectories/generated_platooning.npz')
    
    # vehicle timing in simulations are sampled at 1kHz
    trj_numsamples = 1000 # will correspond to 1s
    
    # simulation timing parameters, generate simulation clocks
    # define a special "vehicle clock" to follow the trajectory
    f_vehicle = 1e3;         # [Hz], trajectory simulation clock rate, fixed by trajectory generation
    t_vehicle = 1/f_vehicle; # [s] , trajectory simulation clock period, fixed by trajectory generation
    
    t_sim_stop = t_vehicle*trj_numsamples
    s_veh_clock = generate_simulation_clocks(t_sim_stop, sim.f_simulation, f_vehicle, gen_clocked_only=True)
    
    localization_decimation_rate = 10 # to get VLP rate = 100 Hz
    localization_rate = 1/(t_vehicle*localization_decimation_rate)
    localization_num_steps = int(trj_numsamples/localization_decimation_rate)
    
    s_adc_clock = generate_simulation_clocks(t_sim_stop, sim.f_simulation, sim.f_adc_clock, gen_clocked_only=True)
    
    # sample in normal rate
    xRL_to_rxL = a['x_RL'][0:trj_numsamples]; yRL_to_rxL = a['y_RL'][0:trj_numsamples]; 
    xRR_to_rxL = a['x_RR'][0:trj_numsamples]; yRR_to_rxL = a['y_RR'][0:trj_numsamples]
    yRL_to_rxR = a['y_RL'][0:trj_numsamples]; yRR_to_rxR = a['y_RR'][0:trj_numsamples]; 
    hdg  = a['hdg'][0:trj_numsamples];
    xRL_to_rxR = a['x_RL'][0:trj_numsamples] - L_ego;
    xRR_to_rxR = a['x_RR'][0:trj_numsamples] - L_ego;
    
    ### simulate received power
    pwr_txL_to_rxL = received_power(xRL_to_rxL, yRL_to_rxL, z+np.zeros_like(yRL_to_rxL), hdg, sim)
    pwr_txL_to_rxR = received_power(xRL_to_rxR, yRL_to_rxR, z+np.zeros_like(yRL_to_rxR), hdg, sim)
    pwr_txR_to_rxL = received_power(xRR_to_rxL, yRR_to_rxL, z+np.zeros_like(yRR_to_rxL), hdg, sim)
    pwr_txR_to_rxR = received_power(xRR_to_rxR, yRR_to_rxR, z+np.zeros_like(yRR_to_rxR), hdg, sim)
    
    shared_pwr_txL_to_rxL = quad_distribute_power(xRL_to_rxL, yRL_to_rxL, z, sim.rx_config_bundle["f_QRX"], pwr_txL_to_rxL)
    shared_pwr_txL_to_rxR = quad_distribute_power(xRL_to_rxR, yRL_to_rxR, z, sim.rx_config_bundle["f_QRX"], pwr_txL_to_rxR)
    shared_pwr_txR_to_rxL = quad_distribute_power(xRR_to_rxL, yRR_to_rxL, z, sim.rx_config_bundle["f_QRX"], pwr_txR_to_rxL)
    shared_pwr_txR_to_rxR = quad_distribute_power(xRR_to_rxR, yRR_to_rxR, z, sim.rx_config_bundle["f_QRX"], pwr_txR_to_rxR)
    
    delay_txL_to_rxL = propagation_delay(xRL_to_rxL, yRL_to_rxL, sim.lightspeed)
    delay_txL_to_rxR = propagation_delay(xRL_to_rxR, yRL_to_rxR, sim.lightspeed)
    delay_txR_to_rxL = propagation_delay(xRR_to_rxL, yRR_to_rxL, sim.lightspeed)
    delay_txR_to_rxR = propagation_delay(xRR_to_rxR, yRR_to_rxR, sim.lightspeed)
    
    ### run simulation over the list of points
    aoa_rxL_txL     = np.zeros((localization_num_steps)); aoa_rxR_txL     = np.zeros((localization_num_steps))
    aoa_rxL_txR     = np.zeros((localization_num_steps)); aoa_rxR_txR     = np.zeros((localization_num_steps))
    d_rxL_txL       = np.zeros((localization_num_steps)); d_rxR_txL       = np.zeros((localization_num_steps))
    d_rxL_txR       = np.zeros((localization_num_steps)); d_rxR_txR       = np.zeros((localization_num_steps))
    
    cls_x_directbearing     = np.zeros((localization_num_steps, iterations)); 
    cls_y_directrange       = np.zeros((localization_num_steps, iterations));
    
    for j in tqdm(range(0,iterations)):
        for i in range(0, localization_num_steps):
            smp_lo = i*localization_decimation_rate
            smp_hi = (i+1)*localization_decimation_rate
    
            step_time       = s_veh_clock[smp_lo:smp_hi]
            simulation_time = s_adc_clock[int(i*sim.f_adc_clock/f_vehicle):int((i+1)*sim.f_adc_clock/f_vehicle)]
    
            rxLL, rxLR, rxRL, rxRR, delays = gen_qrx_onlyclocked(shared_pwr_txL_to_rxL, shared_pwr_txL_to_rxR, 
                                                                 shared_pwr_txR_to_rxL, shared_pwr_txR_to_rxR,
                                                                 delay_txL_to_rxL, delay_txL_to_rxR, 
                                                                 delay_txR_to_rxL, delay_txR_to_rxR, sim, step_time, 
                                                                 simulation_time, smp_lo, smp_hi);
            (rxLA_txL, rxLB_txL, rxLC_txL, rxLD_txL) = rxLL
            (rxLA_txR, rxLB_txR, rxLC_txR, rxLD_txR) = rxLR
            (rxRA_txL, rxRB_txL, rxRC_txL, rxRD_txL) = rxRL
            (rxRA_txR, rxRB_txR, rxRC_txR, rxRD_txR) = rxRR 
            (delayLL_sigTime, delayLR_sigTime, delayRL_sigTime, delayRR_sigTime) = delays
    
            del rxLL, rxLR, rxRL, rxRR, delays 
    
            # initial transmitted signal is known (mark the +, that's because the signal does a roundtrip
            # which is 1 delay earlier than echo transmission from target vehicle)
            wav_rxL_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time + delayLL_sigTime))
            wav_rxR_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time + delayLR_sigTime))
    
            d_rxL_txL[i] = measure_range_roberts(rxLA_txL + rxLB_txL + rxLC_txL + rxLD_txL, wav_rxL_txL, sim.lightspeed, sim.f_emitted)
            d_rxR_txL[i] = measure_range_roberts(rxRA_txL + rxRB_txL + rxRC_txL + rxRD_txL, wav_rxR_txL, sim.lightspeed, sim.f_emitted)
    
            thd = 1e-5; # just to avoid messing up the graphs when aoa detection is too bad.
            # assume that the VLC subsystem correctly decodes the signal (mark that the delayed signal 
            # is decoded, so the delay isn't actually measured, it's implicitly present on the RX signal)
            wav_rxL_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time - delayLL_sigTime))
            wav_rxR_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time - delayLR_sigTime))
            aoa_rxL_txL[i] = measure_bearing(rxLA_txL, rxLB_txL, rxLC_txL, rxLD_txL, wav_rxL_txL, sim.rx_config_bundle["f_QRX"], thd)
            aoa_rxR_txL[i] = measure_bearing(rxRA_txL, rxRB_txL, rxRC_txL, rxRD_txL, wav_rxR_txL, sim.rx_config_bundle["f_QRX"], thd)
    
            cls_x_directbearing[i,j], _ = classicalfix_directbearing_mle(aoa_rxL_txL[i], aoa_rxR_txL[i], L_ego)
            _, cls_y_directrange[i,j]   = classicalfix_directrange_mle(d_rxL_txL[i], d_rxR_txL[i], L_ego)

    return cls_x_directbearing, cls_y_directrange, (localization_decimation_rate, localization_num_steps, 
                                                    xRL_to_rxL, yRL_to_rxL, hdg, s_veh_clock)

### Run simulation for different configurations

different speeds, lambertian and z=0

In [4]:
est_x_lmb20, est_y_lmb20, locpkg   = simrun(sim_lmb20, 0, iterations);
est_x_lmb50, est_y_lmb50,_   = simrun(sim_lmb50, 0, iterations);
est_x_pcwlmb, est_y_pcwlmb,_ = simrun(sim_pcwlmb, 0, iterations);
est_x_real, est_y_real,_     = simrun(sim_real, 0, iterations);

est_x_lmb20_z0p15, est_y_lmb20_z0p15,_ = simrun(sim_lmb20, 0.15, iterations);
est_x_lmb20_z0p30, est_y_lmb20_z0p30,_ = simrun(sim_lmb20, 0.30, iterations);
est_x_lmb20_z0p45, est_y_lmb20_z0p45,_ = simrun(sim_lmb20, 0.45, iterations);
est_x_lmb20_z0p60, est_y_lmb20_z0p60,_ = simrun(sim_lmb20, 0.60, iterations);

localization_decimation_rate = locpkg[0]
localization_num_steps       = locpkg[1]
xRL_to_rxL                   = locpkg[2]
yRL_to_rxL                   = locpkg[3]
hdg                          = locpkg[4]
s_veh_clock                  = locpkg[5]

  0%|                                                  | 0/2000 [00:00<?, ?it/s]/home/ubuntu/vehicular-vlp-simulations/positioning.py:108: RuntimeWarning: invalid value encountered in sqrt
  est_y = np.sqrt(dL**2-est_x**2)
/home/ubuntu/vehicular-vlp-simulations/positioning.py:102: RuntimeWarning: divide by zero encountered in scalar divide
  est_x = L*(1 + np.sin(np.deg2rad(aoaR))*np.cos(np.deg2rad(aoaL))/np.sin(np.deg2rad(aoaL - aoaR)));
/home/ubuntu/vehicular-vlp-simulations/positioning.py:103: RuntimeWarning: divide by zero encountered in scalar divide
  est_y = L*np.cos(np.deg2rad(aoaR))*np.cos(np.deg2rad(aoaL))/np.sin(np.deg2rad(aoaL - aoaR));
  0%|                                                  | 0/2000 [00:00<?, ?it/s]/home/ubuntu/vehicular-vlp-simulations/positioning.py:108: RuntimeWarning: invalid value encountered in sqrt
  est_y = np.sqrt(dL**2-est_x**2)
/home/ubuntu/vehicular-vlp-simulations/positioning.py:102: RuntimeWarning: divide by zero encountered in scalar divide
 

### Save

In [5]:
np.savez("results/survey_es7.npz", 
         iterations = iterations,
         ldr = localization_decimation_rate,
         lns = localization_num_steps,
         xRL_to_rxL=xRL_to_rxL, yRL_to_rxL=yRL_to_rxL, hdg=hdg,
         s_veh_clock = s_veh_clock,
         est_x_lmb20 = est_x_lmb20,
         est_y_lmb20 = est_y_lmb20,
         est_x_lmb50 = est_x_lmb50,
         est_y_lmb50 = est_y_lmb50,
         est_x_pcwlmb = est_x_pcwlmb,
         est_y_pcwlmb = est_y_pcwlmb,
         est_x_real = est_x_real,
         est_y_real = est_y_real,
         est_x_lmb20_z0p15=est_x_lmb20_z0p15,
         est_y_lmb20_z0p15=est_y_lmb20_z0p15,
         est_x_lmb20_z0p30=est_x_lmb20_z0p30,
         est_y_lmb20_z0p30=est_y_lmb20_z0p30,
         est_x_lmb20_z0p45=est_x_lmb20_z0p45,
         est_y_lmb20_z0p45=est_y_lmb20_z0p45,
         est_x_lmb20_z0p60=est_x_lmb20_z0p60,
         est_y_lmb20_z0p60=est_y_lmb20_z0p60)